In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
import json
import base64
import os

## Data Loading

In [2]:

df = pd.read_csv(os.path.join('data', 'data_clean.csv'))

In [3]:
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])

In [4]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

## Predictions

In [9]:
def create_tf_example(input_data):
    feature = {}
    for key, value in input_data.items():
        if isinstance(value, int):
            feature[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, float):
            feature[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, str):
            feature[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))
        else:
            raise ValueError(f"Unsupported data type for key '{key}': {type(value)}")
    
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

pred = df.drop(['diagnosis'], axis=1).sample(1).to_dict(orient='records')[0]

serialized_example = create_tf_example(pred)

In [ ]:
json_data = json.dumps({
    "instances": [
        {
            "examples": {
                "b64": base64.b64encode(serialized_example).decode('utf-8')
            }
        }
    ]
})

endpoint = "https://mlops02-breast-cancer-pred-6dad42a70615.herokuapp.com/v1/models/breast_cancer_wisconsin_model/versions/1753890624:predict" 

try:
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    response.raise_for_status()
    
    prediction = response.json()["predictions"][0][0]
    
    print(f"Prediction raw score: {prediction:.4f}")

    if prediction >= 0.5:
        print('Malignant (Cancer detected)')
    else:
        print('Benign (No cancer detected)')

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    if response is not None:
        print(f"Response content: {response.content.decode('utf-8')}")
except KeyError:
    print("Unexpected response format. Check the structure of response.json().")
    print(f"Full response: {response.json()}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Prediction raw score: 1.0000
Malignant (Cancer detected)
